# Number 4
4.	Anda diberikan sebuah dataset tentang data kelulusan mahasiswa yang dapat diambil pada link .... . Jika diperhatikan dataset ini adalah dataset yang imbalance.
a. [5%] Berikan Solusi terhadap data imbalaced yang ditemui
b. [10%] Gunakan model klasifikasi Decision Tree untuk data tersebut
c. [5%] Lakukan evaluasi performa model klasifikasi yang dibangun

Before proceeding to solve the problem, I basically taking a look at the data wise. The data is barely balanced. Not just the label tho'. The feature are also seems to be, imbalanced. Especially when it comes to the ordinal / categorical data. So what I planning is:

1. Data Pre-Processing
- Drop "NAMA" column since it is not really matters.
- Drop any row that contains NaN or null data. 
- Perform Hot-Top Encoding to "JENIS KELAMIN", "STATUS MAHASISWA", "STATUS NIKAH".
- Convert "STATUS KELULUSAN", so that "TEPAT" converted to 1, otherwise goes 0.

2. "Fixing" the Imbalance problem
- Perform the impossible by oversampling the data label
- Perform data normalization by performing data scaler to all features.

3. Training Preparation
- Train-Test Split

4. Model Fitting
- Parameter Searches
- Model Fitting

5. Model Evaluation
- Moddel Evaluation with Accuracy and ROC AUC

In [1]:
import pandas;
from imblearn.over_sampling import SMOTE;
from sklearn.preprocessing import StandardScaler;
from sklearn.model_selection import train_test_split;

from sklearn.tree import DecisionTreeClassifier;
from sklearn.model_selection import GridSearchCV;
from sklearn.metrics import roc_auc_score, accuracy_score;

## Data Exploration Part

Basically just head the code out to show the data.

In [2]:
dataframe = pandas.read_csv("datakelulusanmahasiswa-imbalance.csv");
dataframe.head()

,NAMA,JENIS KELAMIN,STATUS MAHASISWA,UMUR,STATUS NIKAH,IPS 1,IPS 2,IPS 3,IPS 4,IPS 5,IPS 6,IPS 7,IPS 8,IPK,STATUS KELULUSAN
0,UNAMA,LAKI - LAKI,MAHASISWA,24,BELUM MENIKAH,3.17,2.70,3.23,2.41,3.00,2.47,1.75,0.00,2.75,TEPAT
1,LEYLA TRIYANA PRATIWI,PEREMPUAN,MAHASISWA,26,BELUM MENIKAH,3.60,3.50,3.42,2.85,3.31,2.95,2.18,NaN,3.39,TEPAT
2,VERIS SOFIYAN PRAYOGA,LAKI - LAKI,MAHASISWA,29,BELUM MENIKAH,2.67,2.66,2.93,3.14,2.92,2.64,2.88,0.50,2.81,TEPAT
3,ADITYA AKBAR NUGRAHA,LAKI - LAKI,MAHASISWA,27,BELUM MENIKAH,2.48,2.86,2.09,2.55,2.55,2.43,2.55,2.17,2.82,TEPAT
4,ERNA EKA RIYANTI,PEREMPUAN,MAHASISWA,25,BELUM MENIKAH,3.19,3.08,3.31,2.83,3.36,2.73,3.06,0.00,3.09,TEPAT


## 1. Data Pre-Processing
This part contains data pre-processing for the data. Data cleansing and preparation for the data so it could be processed later, and actually can be feed to the model as a tensor are applied here. 

- Drop "NAMA" column since it is not really matters.
- Drop any row that contains NaN or null data. 
- Perform Hot-Top Encoding to "JENIS KELAMIN", "STATUS MAHASISWA", "STATUS NIKAH".
- Convert "STATUS KELULUSAN", so that "TEPAT" converted to 1, otherwise goes 0.

In [3]:
# Drop name column
dataframe = dataframe.drop(columns = ["NAMA"]);

# Drop any row containing NaN or null data
dataframe = dataframe.dropna();

# Perform Hot-Top Encoding to all ordinal data.
for i in ["JENIS KELAMIN", "STATUS MAHASISWA", "STATUS NIKAH"]:
    hot_encoded = pandas.get_dummies(dataframe[i], prefix = i);
    dataframe = pandas.concat([dataframe, hot_encoded], axis = 1);
    dataframe = dataframe.drop(columns = [i]);

# Convert labels to binary format.
new_label = [];

for i in dataframe["STATUS KELULUSAN"]:
    if(i == "TEPAT"):
        new_label.append(1);
    else:
        new_label.append(0);

dataframe = dataframe.drop(columns = ["STATUS KELULUSAN"]);
dataframe["STATUS KELULUSAN"] = new_label;

# Print the column result of data pre-processing 
dataframe.columns

Index(['UMUR', 'IPS 1', 'IPS 2', 'IPS 3', 'IPS 4', 'IPS 5', 'IPS 6', 'IPS 7',
       'IPS 8', 'IPK ', 'JENIS KELAMIN_LAKI - LAKI', 'JENIS KELAMIN_PEREMPUAN',
       'STATUS MAHASISWA_BEKERJA', 'STATUS MAHASISWA_MAHASISWA',
       'STATUS NIKAH_BELUM MENIKAH', 'STATUS KELULUSAN'],
      dtype='object')

## 2. "Fixing" the Imbalance problem

Technically, when we face data imbalance, the problem are came from the data provider. But, since we can blame no one here, I will "fix" the imbalance problem anyway. The problem with the dataset is, that there are 136 rows of data with label of "1" while only 2 with label of "0". Well I can use SVM OneClass, but unfortunately, they want me to solve the problem with Decision Tree, so I cannot just drop the 2 negative clause data. So there are two treatments are applied to the data:

- Label set: Perform the impossible by oversampling employing SMOTE by imblearn. 
This make new augmented data rows using statistical analysis performed within SMOTE, in which, in this case, are generating the data with label "0", as much as possible, so it is considered as "balanced" by ratio. 

- Feature set: Perform data normalization by performing data scaler to all features.  
This treatment is to make every single features columns are standardized, by lowering the standard deviation, and make the value mean of 0. This will ease the model training phase. 

In [4]:
# SMOTE operation

print("Total data before SMOTE: ", len(dataframe["STATUS KELULUSAN"]))

oversample = SMOTE(random_state = 42, k_neighbors = 1);

feature_resampled, label_resampled = oversample.fit_resample(
    dataframe.drop(columns = ["STATUS KELULUSAN"]), 
    dataframe["STATUS KELULUSAN"],
);

generated_data = len(label_resampled);

feature_resampled["STATUS KELULUSAN"] = label_resampled;

dataframe = pandas.concat([dataframe, feature_resampled], ignore_index = True);

print("Total augmented data: ", len(label_resampled));

print("Total after concatenated with augmented data: ", len(dataframe));

Total data before SMOTE:  138
Total augmented data:  272
Total after concatenated with augmented data:  410


In [5]:
# SMOTE result
print(dataframe["STATUS KELULUSAN"].value_counts());

STATUS KELULUSAN
1    272
0    138
Name: count, dtype: int64


In [6]:
# Standard Scaler

features = dataframe.drop(columns = ["STATUS KELULUSAN"]);

scaler = StandardScaler();
features = scaler.fit_transform(features);

## 3. Training Preparation

This part contains the training preparation. This part conducts a train-test data splitting.

| Data Split | Ratio |
|---|---|
| Train | 70% |
| Test | 30% |

In [7]:
feature_train, feature_test, label_train, label_test = train_test_split(
    features, 
    dataframe["STATUS KELULUSAN"],
    test_size = 0.3,
    train_size = 0.7,
    random_state = 42
);

## 4. Model Training

This part contains the model training process using sklearn.tree.DecisionTreeClassifier.
The plan is, by employing GridSearchCV, what I do is basically experimenting with several parameters as below:

| Parameter | Value being tested |
|---|---|
| `criterion` | `gini`, `entropy`, `log_loss` |
| `splitter` | `best`, `random` |

To decide which parameters are good, `roc_auc` is employed as its model evaluation. Also, within training process, I also do cross-validation with 5-Fold Cross Validation.

See [this link regarding DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for more information.

In [8]:
# Define parameters grid as mentioned above
parameters = {
    "criterion": ["gini", "entropy", "log_loss"],
    "splitter": ["best", "random"]
};

model = DecisionTreeClassifier();

# cv = 5 means 5-Fold Cross Validations
searcher = GridSearchCV(model, param_grid = parameters, cv = 5, scoring = "roc_auc");

searcher.fit(feature_train, label_train);

best_parameters = searcher.best_params_;

# This is where the best model for prediction is defined.
best_model = searcher.best_estimator_;

print(f"The best parameters searched is {best_parameters}");

The best parameters searched is {'criterion': 'gini', 'splitter': 'best'}


## 5. Model Evaluation

This part contains the model evaluation. Basically, taking the best model from `best_model` variable aboves, and then testing it with `accuracy` and `roc_auc_score` function.

In [9]:
prediction = best_model.predict(feature_test);

accuracy = accuracy_score(label_test, prediction);
roc_auc = roc_auc_score(label_test, prediction);

print(f"Accuracy: {accuracy}\nROC AUC: {roc_auc}");

Accuracy: 0.983739837398374
ROC AUC: 0.9880952380952381
